In [287]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import date, time, datetime

In [288]:
train_users = pd.read_csv("train_users.csv")
test_users = pd.read_csv("test_users.csv")
sessions = pd.read_csv("sessions.csv")

In [289]:
train_users.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [290]:
train_users.drop(['country_destination'], axis = 1)
#Concatenating train and test data for EDA
df_all = pd.concat((train_users, test_users), axis = 0, ignore_index= True)

C:\Users\owenf\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


In [291]:
dates_list = []
year_list = []
month_list = []

for i in df_all["date_account_created"]:
    date_time_obj = datetime.strptime(i, '%Y-%m-%d')
    year = date_time_obj.date().year
    month = date_time_obj.date().month
    dates_list.append(date_time_obj)
    year_list.append(year)
    month_list.append(month)

df_all["datetime_account_created"] = dates_list
df_all["year_account_created"] = year_list
df_all["month_account_created"] = month_list

In [292]:
dates_list = []
year_list = []
month_list = []

for i in df_all["timestamp_first_active"]:
    time_stamp_obj = pd.to_datetime(i , format='%Y%m%d%H%M%S')
    year = time_stamp_obj.date().year
    month = time_stamp_obj.date().month
    dates_list.append(time_stamp_obj)
    year_list.append(year)
    month_list.append(month)
    
df_all["datetime_first_active"] = dates_list
df_all["year_first_active"] = year_list
df_all["month_first_active"] = month_list

In [293]:
df_all["active_after_created"] = df_all["datetime_first_active"] - df_all["datetime_account_created"]

In [294]:
df_all['age'].quantile(0.5)

33.0

In [295]:
df_all['age'] = np.where(df_all['age'] > 110, 34, df_all['age'])
df_all['age'].describe()

count    158681.000000
mean         36.838115
std          13.766690
min           1.000000
25%          28.000000
50%          33.000000
75%          42.000000
max         110.000000
Name: age, dtype: float64

In [296]:
by_signup_device = df_all.groupby(['signup_method', 'first_device_type'])

In [297]:
def impute_median(series):
    return series.fillna(series.median())

In [298]:
df_all.age = by_signup_device['age'].transform(impute_median)

In [300]:
df_all['first_affiliate_tracked'].value_counts()

untracked        143181
linked            62064
omg               54859
tracked-other      6655
product            2353
marketing           281
local ops            69
Name: first_affiliate_tracked, dtype: int64

In [301]:
tracked = []

for i in df_all['first_affiliate_tracked']:
    if i == "untracked":
        isTracked = 1
    else:
        isTracked = 0
    tracked.append(isTracked)

df_all['is_first_affiliate_tracked'] = tracked

In [302]:
df_all.isnull().sum()

affiliate_channel                  0
affiliate_provider                 0
age                                1
country_destination            62096
date_account_created               0
date_first_booking            186639
first_affiliate_tracked         6085
first_browser                      0
first_device_type                  0
gender                             0
id                                 0
language                           0
signup_app                         0
signup_flow                        0
signup_method                      0
timestamp_first_active             0
datetime_account_created           0
year_account_created               0
month_account_created              0
datetime_first_active              0
year_first_active                  0
month_first_active                 0
active_after_created               0
is_first_affiliate_tracked         0
dtype: int64

In [303]:
df_all.head()

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,...,signup_method,timestamp_first_active,datetime_account_created,year_account_created,month_account_created,datetime_first_active,year_first_active,month_first_active,active_after_created,is_first_affiliate_tracked
0,direct,direct,33.0,NDF,2010-06-28,NaN,untracked,Chrome,Mac Desktop,-unknown-,...,facebook,20090319043255,2010-06-28,2010,6,2009-03-19 04:32:55,2009,3,-466 days +04:32:55,1
1,seo,google,38.0,NDF,2011-05-25,NaN,untracked,Chrome,Mac Desktop,MALE,...,facebook,20090523174809,2011-05-25,2011,5,2009-05-23 17:48:09,2009,5,-732 days +17:48:09,1
2,direct,direct,56.0,US,2010-09-28,2010-08-02,untracked,IE,Windows Desktop,FEMALE,...,basic,20090609231247,2010-09-28,2010,9,2009-06-09 23:12:47,2009,6,-476 days +23:12:47,1
3,direct,direct,42.0,other,2011-12-05,2012-09-08,untracked,Firefox,Mac Desktop,FEMALE,...,facebook,20091031060129,2011-12-05,2011,12,2009-10-31 06:01:29,2009,10,-765 days +06:01:29,1
4,direct,direct,41.0,US,2010-09-14,2010-02-18,untracked,Chrome,Mac Desktop,-unknown-,...,basic,20091208061105,2010-09-14,2010,9,2009-12-08 06:11:05,2009,12,-280 days +06:11:05,1


In [ ]:
# The rest of this code was my failed attempt to extract a secs_elapsed feature. Everything I look at points to left join
# having all of the values that inner join would have plus NAN values if the user does not have a session. But, left join
# shows all NAN values and inner join gets rid of a lot of rows.

In [304]:
df_all_users = df_all.rename(columns={"id":"user_id"})

In [305]:
sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [306]:
seconds = sessions.groupby('user_id', as_index=False).agg({"secs_elapsed": "sum"})
seconds.head()

,user_id,secs_elapsed
0,0035hobuyj,5724670.0
1,007gj7kqdk,370406.0
2,009a40t3dk,2079299.0
3,00allnceb8,0.0
4,00fhpdik5t,221087.0


In [311]:
df_sessions_inner = pd.merge(df_all_users, seconds, on="user_id")

In [312]:
df_sessions_inner

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,...,timestamp_first_active,datetime_account_created,year_account_created,month_account_created,datetime_first_active,year_first_active,month_first_active,active_after_created,is_first_affiliate_tracked,secs_elapsed
0,sem-non-brand,google,62.0,other,2014-01-01,2014-01-04,omg,Chrome,Windows Desktop,MALE,...,20140101000936,2014-01-01,2014,1,2014-01-01 00:09:36,2014,1,00:09:36,0,3427529.0
1,direct,direct,34.0,NDF,2014-01-01,NaN,untracked,Firefox,Mac Desktop,-unknown-,...,20140101001558,2014-01-01,2014,1,2014-01-01 00:15:58,2014,1,00:15:58,1,207842.0
2,sem-brand,google,34.0,NDF,2014-01-01,NaN,omg,Firefox,Windows Desktop,-unknown-,...,20140101001639,2014-01-01,2014,1,2014-01-01 00:16:39,2014,1,00:16:39,0,1135444.0
3,direct,direct,34.0,NDF,2014-01-01,NaN,linked,Chrome,Windows Desktop,-unknown-,...,20140101002146,2014-01-01,2014,1,2014-01-01 00:21:46,2014,1,00:21:46,0,3755100.0
4,direct,direct,31.0,GB,2014-01-01,2014-01-02,untracked,-unknown-,iPhone,-unknown-,...,20140101002619,2014-01-01,2014,1,2014-01-01 00:26:19,2014,1,00:26:19,1,2555.0
5,direct,direct,38.0,NDF,2014-01-01,NaN,linked,Mobile Safari,iPad,-unknown-,...,20140101002626,2014-01-01,2014,1,2014-01-01 00:26:26,2014,1,00:26:26,0,1380288.0
6,seo,google,32.0,US,2014-01-01,2014-01-07,linked,Mobile Safari,iPad,FEMALE,...,20140101002742,2014-01-01,2014,1,2014-01-01 00:27:42,2014,1,00:27:42,0,49673.0
7,direct,direct,34.0,NDF,2014-01-01,NaN,untracked,Chrome,Windows Desktop,-unknown-,...,20140101003535,2014-01-01,2014,1,2014-01-01 00:35:35,2014,1,00:35:35,1,605413.0
8,sem-brand,google,19.0,other,2014-01-01,2014-12-04,untracked,Safari,Mac Desktop,-unknown-,...,20140101005503,2014-01-01,2014,1,2014-01-01 00:55:03,2014,1,00:55:03,1,1284401.0
9,direct,direct,25.0,NDF,2014-01-01,NaN,untracked,Chrome,Mac Desktop,MALE,...,20140101005837,2014-01-01,2014,1,2014-01-01 00:58:37,2014,1,00:58:37,1,2868205.0


In [313]:
df_sessions_left = pd.merge(df_all_users, seconds, on="user_id", how='left')

In [314]:
df_sessions_left

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,...,timestamp_first_active,datetime_account_created,year_account_created,month_account_created,datetime_first_active,year_first_active,month_first_active,active_after_created,is_first_affiliate_tracked,secs_elapsed
0,direct,direct,33.0,NDF,2010-06-28,NaN,untracked,Chrome,Mac Desktop,-unknown-,...,20090319043255,2010-06-28,2010,6,2009-03-19 04:32:55,2009,3,-466 days +04:32:55,1,NaN
1,seo,google,38.0,NDF,2011-05-25,NaN,untracked,Chrome,Mac Desktop,MALE,...,20090523174809,2011-05-25,2011,5,2009-05-23 17:48:09,2009,5,-732 days +17:48:09,1,NaN
2,direct,direct,56.0,US,2010-09-28,2010-08-02,untracked,IE,Windows Desktop,FEMALE,...,20090609231247,2010-09-28,2010,9,2009-06-09 23:12:47,2009,6,-476 days +23:12:47,1,NaN
3,direct,direct,42.0,other,2011-12-05,2012-09-08,untracked,Firefox,Mac Desktop,FEMALE,...,20091031060129,2011-12-05,2011,12,2009-10-31 06:01:29,2009,10,-765 days +06:01:29,1,NaN
4,direct,direct,41.0,US,2010-09-14,2010-02-18,untracked,Chrome,Mac Desktop,-unknown-,...,20091208061105,2010-09-14,2010,9,2009-12-08 06:11:05,2009,12,-280 days +06:11:05,1,NaN
5,other,other,34.0,US,2010-01-01,2010-01-02,omg,Chrome,Mac Desktop,-unknown-,...,20100101215619,2010-01-01,2010,1,2010-01-01 21:56:19,2010,1,0 days 21:56:19,0,NaN
6,other,craigslist,46.0,US,2010-01-02,2010-01-05,untracked,Safari,Mac Desktop,FEMALE,...,20100102012558,2010-01-02,2010,1,2010-01-02 01:25:58,2010,1,0 days 01:25:58,1,NaN
7,direct,direct,47.0,US,2010-01-03,2010-01-13,omg,Safari,Mac Desktop,FEMALE,...,20100103191905,2010-01-03,2010,1,2010-01-03 19:19:05,2010,1,0 days 19:19:05,0,NaN
8,other,craigslist,50.0,US,2010-01-04,2010-07-29,untracked,Safari,Mac Desktop,FEMALE,...,20100104004211,2010-01-04,2010,1,2010-01-04 00:42:11,2010,1,0 days 00:42:11,1,NaN
9,other,craigslist,46.0,US,2010-01-04,2010-01-04,omg,Firefox,Mac Desktop,-unknown-,...,20100104023758,2010-01-04,2010,1,2010-01-04 02:37:58,2010,1,0 days 02:37:58,0,NaN
